<h1><center>Build <a href="https://howsmynhs.co.uk/">HowsMyNHS</a> website</center></h1>

This is the main script used to automatically generate the html for <a href="https://howsmynhs.co.uk/">HowsMyNHS</a>.

In [1]:
import build_website as bw
waiting_data = 'NHSwaiting.npy'
bed_data = 'NHSbeds.npy'

<h2>Generate Plots</h2>

In [2]:
#bw.plotWaitingData(waiting_data)
#bw.plotBedData(bed_data)

<h2>Make homepage</h2>

In [3]:
bw.MakeHomepage(waiting_data, bed_data)

Building homepage... Done


<h2>Make all the other pages</h2>

In [4]:
bw.build_trust_pages(waiting_data, bed_data)

Building trust pages... England
Bedfordshire Hospitals NHS Foundation Trust
Cambridge University Hospitals NHS Foundation Trust
Clacton Hospital
East And North Hertfordshire NHS Trust
East Suffolk And North Essex NHS Foundation Trust
Fryatt Hospital
Hertfordshire Community NHS Trust
Herts Urgent Care (Ascots Lane)
James Paget University Hospitals NHS Foundation Trust
Mid And South Essex NHS Foundation Trust
Milton Keynes University Hospital NHS Foundation Trust
Norfolk And Norwich University Hospitals NHS Foundation Trust
North West Anglia NHS Foundation Trust
Putnoe Medical Centre Walk In Centre
The Princess Alexandra Hospital NHS Trust
The Queen Elizabeth Hospital, King's Lynn, NHS Foundation Trust
West Hertfordshire Hospitals NHS Trust
West Suffolk NHS Foundation Trust
Barking, Havering And Redbridge University Hospitals NHS Trust
Barts Health NHS Trust
Beckenham Beacon Ucc
Central London Community Healthcare NHS Trust
Chelsea And Westminster Hospital NHS Foundation Trust
Croydon He

Torbay And South Devon NHS Foundation Trust
University Hospitals Bristol And Weston NHS Foundation Trust
University Hospitals Plymouth NHS Trust
Wiltshire Health & Care
Yeovil District Hospital NHS Foundation Trust
Bedford Hospital NHS Trust
Luton And Dunstable University Hospital NHS Foundation Trust
West London NHS Trust
Great Ormond Street Hospital For Children NHS Foundation Trust
The Royal Marsden NHS Foundation Trust
South West London And St George's Mental Health NHS Trust
Barnet, Enfield And Haringey Mental Health NHS Trust
Central And North West London NHS Foundation Trust
East London NHS Foundation Trust
Camden And Islington NHS Foundation Trust
Avon And Wiltshire Mental Health Partnership NHS Trust
Devon Partnership NHS Trust
Solent NHS Trust
Portsmouth Hospitals NHS Trust
Oxleas NHS Foundation Trust
Sussex Partnership NHS Foundation Trust
Surrey And Borders Partnership NHS Foundation Trust
Kent And Medway NHS And Social Care Partnership Trust
Worcestershire Health And Care 